## Natural Language Processing
## Final Project - Karan Mehta

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import re
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import multiprocessing

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

C:\Users\Karan Mehta\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
%%time
df_news_final_project = pd.read_parquet("D:/University of Chicago/Courses/ADSP 32018 IP02 (Winter 2024) Natural Language Processing and Cognitive Computing/Final Project/news_final_project.parquet", engine='pyarrow')
df_news_final_project.shape

CPU times: total: 5.45 s
Wall time: 16.7 s


(200435, 5)

In [3]:
df_news_final_project.head(10)

,url,date,language,title,text
0,http://galusaustralis.com/2020/02/486473/legaltech-artificial-intelligence-market-2019-technology-advancement-and-future-scope-casetext-inc-catalyst-repository-systems-ebrevia/,2020-02-26,en,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis \n\nGalus Australis\n\nBusine..."
1,http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/,2020-02-27,en,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament,"\nChildren With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament\n \n\nSkip to content\n\t\t\tThursday, February 27, 2020\t\t\n\nLatest:\n\n\n..."
2,http://www.agoravox.it/Covid-19-un-messaggio-dai.html,2020-03-13,en,Covid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia,\n\n\nCovid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia\n\nAccedi\n\nPartecipa ad AgoraVox!\nIscriviti e proponi un articolo\nHome page\n\n\nAttualitÃ \n\n\nA...
3,http://www.desototimes.com/news/child-actor-claude-jarman-jr-looks-back-on-his-movie-career-during-the-golden-days/article_0b310fae-e767-11ec-982e-a380305dda2f.html,2022-06-09,en,Child actor Claude Jarman Jr. looks back on his movie career during the “Golden Days” of Hollywood | News | desototimes.com,\nChild actor Claude Jarman Jr. looks back on his movie career during the “Golden Days” of Hollywood | News | desototimes.com\nSkip to main content\n\nYou have permission to edit this article.\n\n...
4,http://www.millenniumpost.in/big-stories/ai-express-aircraft-skids-off-runaway-in-kozhikode-rescue-ops-on-414926,2020-08-07,en,Two dead as AI Express flight skids off Kozhikode airport,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGl...
5,http://www.nativestew.com/2023/11/ai-took-my-job.html,2023-11-20,en,"Native Stew - Bahamas AI Art, Photos, Videos: AI Took My Job","\nNative Stew - Bahamas AI Art, Photos, Videos: AI Took My Job\nPages\nHome\n\n\nPeople\n\n\nPlaces\n\n\nThings\n\nMonday, November 20, 2023\n\nAI Took My Job\n ""AI Took My Job"" - AI art©A. Derek ..."
6,http://www.pentictonherald.ca/entertainment/national_entertainment/article_33071177-695b-5c1c-bccf-f24c3ca132f0.html,2020-12-08,en,From the Bard to broadcaster: Stratford Festival builds new identity with streamer | National Entertainment | pentictonherald.ca,\n\nFrom the Bard to broadcaster: Stratford Festival builds new identity with streamer | National Entertainment | pentictonherald.ca\n\nYou have permission to edit this article.\n\n Edit\nClose\n\...
7,http://www.peripherals.consumerelectronicsnet.com/multicoreware-inc-becomes-cevas-trusted-partner-for-imaging-computer-vision/,2021-02-25,en,MulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net\n\n \nSkip to content\n\nConsumer Electronics Net\n\nPrimary Menu\n\nConsumer Electro...
8,http://www.sbwire.com/press-releases/healthcare-artificial-intelligence-market-next-big-thing-major-giants-general-electric-medtronic-aicure-apixio-1363690.htm,2022-10-06,en,Healthcare Artificial Intelligence Market Analysis and Forecast for Next 5 Years,\n\nHealthcare Artificial Intelligence Market Analysis and Forecast for Next 5 Years\n \n\n \n\nSBWire\nSign Up\nLogin\n\nOur Service\nPlans & Pricing\nNewsroom\nHelp\nAbout\n\nAMA Research &amp; 

In [4]:
df_news_final_project['date'] = pd.to_datetime(df_news_final_project['date'])
df_news_final_project['year'] = df_news_final_project['date'].dt.year

In [5]:
df_news_final_project['year'].value_counts()

year
2023    119663
2022     30083
2021     24094
2020     18948
2024      7647
Name: count, dtype: int64

### News Article (Text) Cleaning

In [6]:
num_processors = multiprocessing.cpu_count()
num_processors

workers = num_processors-1

print(f'Using {workers} workers')

Using 11 workers


In [7]:
%%time
# Removing urls
df_news_final_project['cleaned'] = df_news_final_project['text'].replace(to_replace=r'\b(?:https?://)?(?:www\.)?\w+\.(?:com|net|io|edu|gov|org|co)\b', value='', regex=True)

CPU times: total: 1min 23s
Wall time: 1min 52s


In [8]:
%%time
# Removing new line character
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub(r'(?:[\n]+)',' ',str(x)))

CPU times: total: 22.5 s
Wall time: 28 s


In [9]:
%%time
# Removing tab
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub(r'(?:[\t]+)',' ',str(x)))

CPU times: total: 19.2 s
Wall time: 21.7 s


In [10]:
# Removing HTMl tags
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub('<[^>]*>',' ',str(x)))

In [11]:
# Removing CSS tags
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub('<style[^>]*>.*?</style>',' ',str(x)))

In [12]:
# Removing Javascript tags
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub('<script[^>]*>.*?</script>',' ',str(x)))

In [13]:
# Remove all non-ascii characters
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub(r'[^\x00-\x7F]',' ',str(x)))

In [14]:
# Remove all emojis and symbols characters
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub('[\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\u2600-\u26FF\u2700-\u27BF]',' ',str(x)))

In [15]:
df_news_final_project['cleaned'] = df_news_final_project['cleaned'].apply(lambda x: re.sub('#\w+',' ',str(x)))

### News Article (Title) Cleaning

In [16]:
df_news_final_project['clean_title'] = df_news_final_project['title'].apply(lambda x: re.sub(r'[^\w\s.,]', '', x))

In [17]:
%%time
# Removing 
df_news_final_project['clean_title'] = df_news_final_project['text'].replace(to_replace=r'\b(?:https?://)?(?:www\.)?\w+\.(?:com|net|io|edu|gov|org|co)\b', value='', regex=True)

CPU times: total: 1min 19s
Wall time: 1min 40s


In [18]:
%%time
# Removing new line character
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].apply(lambda x: re.sub(r'(?:[\n]+)',' ',str(x)))

CPU times: total: 25.7 s
Wall time: 36 s


In [19]:
%%time
# Removing tab
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].apply(lambda x: re.sub(r'(?:[\t]+)',' ',str(x)))

CPU times: total: 16.7 s
Wall time: 22.1 s


In [20]:
# Removing HTMl tags
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].apply(lambda x: re.sub('<[^>]*>',' ',str(x)))

In [21]:
# Remove all non-ascii characters
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].apply(lambda x: re.sub(r'[^\x00-\x7F]',' ',str(x)))

In [22]:
# Remove all emojis and symbols characters
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].apply(lambda x: re.sub('[\U0001F300-\U0001F5FF\U0001F600-\U0001F64F\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\u2600-\u26FF\u2700-\u27BF]',' ',str(x)))

In [23]:
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].apply(lambda x: re.sub('#\w+',' ',str(x)))

In [24]:
df_news_final_project['clean_title'] = df_news_final_project['clean_title'].str.strip()

### Finding Relevant Articles

In [25]:
%%time
# Finding all the relevant articles 
key_tokens = ['artificial intelligence', 'machine learning', 'analytics', 'openai', 'chatgpt', 
              'nlp', 'data', 'llm', 'Autoencoders', 'Predictive model']

pattern = '|'.join(map(re.escape, key_tokens))

df_news_final_project['relevant'] = df_news_final_project['cleaned'].str.contains(pattern, flags=re.IGNORECASE).astype(int)

CPU times: total: 28.8 s
Wall time: 45.2 s


In [26]:
df_news_final_project['relevant'].value_counts()

relevant
1    185245
0     15190
Name: count, dtype: int64

In [27]:
df_relevant = df_news_final_project[df_news_final_project['relevant'] == 1]

In [28]:
%%time
# Below words exist at the bottom of most news articles and are not relevant (remnants of the web crawler)
webcrawl_remnants = ['Advertisement', 'Sponsored', 'Copyright', 'Terms of Service', 'Privacy Policy', 'About Us', 
                     'Contact Us', 'Subscribe', 'Share', 'Recommended Reading', 'Follow Us', 'Tag Cloud', 'Archive',
                     'Search', 'Disclaimer', 'Cookies', 'Sitemap', 'Loading', 'Error', 'All rights reserved', 
                     'Newsletter', 'Press Release', 'Banner', 'Login',  'Policy', 'Feedback', 'Join', 'T&C']

pattern = '|'.join(map(re.escape, webcrawl_remnants))

df_relevant['cleaned'] = df_relevant['cleaned'].str.replace(pattern, ' ', regex=True, flags=re.IGNORECASE)

CPU times: total: 7min 34s
Wall time: 11min 41s


In [29]:
df_relevant['cleaned'] = df_relevant['cleaned'].str.strip()

In [30]:
%%time
df_relevant['tokens'] = df_relevant['cleaned'].apply(lambda x: word_tokenize(x))

CPU times: total: 11min 26s
Wall time: 20min 5s


In [31]:
#%%time
#df_relevant['title_token'] = df_relevant['clean_title'].apply(lambda x: word_tokenize(x))

In [32]:
df_relevant['no_tokens'] = df_relevant['tokens'].apply(len)

In [33]:
df_relevant.head(5)

,url,date,language,title,text,year,cleaned,clean_title,relevant,tokens,no_tokens
0,http://galusaustralis.com/2020/02/486473/legaltech-artificial-intelligence-market-2019-technology-advancement-and-future-scope-casetext-inc-catalyst-repository-systems-ebrevia/,2020-02-26,en,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis \n\nGalus Australis\n\nBusine...",2020,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope Casetext Inc., Catalyst Repository Systems, eBREVIA Galus Australis Galus Australis BusinessGene...","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope Casetext Inc., Catalyst Repository Systems, eBREVIA Galus Australis Galus Australis BusinessGene...",1,"[LegalTech, Artificial, Intelligence, Market, 2019, Technology, Advancement, and, Future, Scope, Casetext, Inc., ,, Catalyst, Repository, Systems, ,, eBREVIA, Galus, Australis, Galus, Australis, B...",1087
2,http://www.agoravox.it/Covid-19-un-messaggio-dai.html,2020-03-13,en,Covid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia,\n\n\nCovid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia\n\nAccedi\n\nPartecipa ad AgoraVox!\nIscriviti e proponi un articolo\nHome page\n\n\nAttualitÃ \n\n\nA...,2020,Covid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia Accedi Partecipa ad AgoraVox! Iscriviti e proponi un articolo Home page Attualit Ambiente Cronaca Cronaca ...,Covid-19: un messaggio dai ricercatori italiani ai colleghi stranieri - AgoraVox Italia Accedi Partecipa ad AgoraVox! Iscriviti e proponi un articolo Home page Attualit Ambiente Cronaca Cronaca ...,1,"[Covid-19, :, un, messaggio, dai, ricercatori, italiani, ai, colleghi, stranieri, -, AgoraVox, Italia, Accedi, Partecipa, ad, AgoraVox, !, Iscriviti, e, proponi, un, articolo, Home, page, Attualit...",2455
4,http://www.millenniumpost.in/big-stories/ai-express-aircraft-skids-off-runaway-in-kozhikode-rescue-ops-on-414926,2020-08-07,en,Two dead as AI Express flight skids off Kozhikode airport,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGl...,2020,Two dead as AI Express flight skids off Kozhikode airportTop Toggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGlobal EyeInla...,Two dead as AI Express flight skids off Kozhikode airportTopLoginSubscribeToggle navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday PostBeyond BygoneEpic PowerIn RetrospectGl...,1,"[Two, dead, as, AI, Express, flight, skids, off, Kozhikode, airportTop, Toggle, navigationFeaturesDelhiKolkataNationOpinionEditorialBusinessSportsSunday, PostBeyond, BygoneEpic, PowerIn, Retrospec...",280
7,http://www.peripherals.consumerelectronicsnet.com/multicoreware-inc-becomes-cevas-trusted-partner-for-imaging-computer-vision/,2021-02-25,en,MulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net\n\n \nSkip to content\n\nConsumer Electronics Net\n\nPrimary Menu\n\nConsumer Electro...,2021,MulticoreWare Inc. Becomes CEVA s Trusted Partner for Imaging & Computer Vision Consumer Electronics Net Skip to content Consumer Electronics Net Primary Menu Consumer Electronics Net for: ...,MulticoreWare Inc. Becomes CEVA s Trusted Partner for Imaging & Computer Vision Consumer Electronics Net Skip to content Consumer Electronics Net Primary Menu Consumer 

### Saving the Cleaned Data


In [34]:
df_relevant.to_parquet('CleanedDataset.parquet')